逆問題を解いてみよう
===
by ほげにむし

逆問題の例として、参考文献[1]に掲載されている以下のような問題を考えてみよう。


魔法陣逆問題
---
---
9個のセルが以下のように並んでいる。9個のセルの中身を直接見ることはできないが、
セルの集まりの、水平、垂直、対角線方向、の和を求めることはできるとする。


$$ \begin{array}{ccc} x_1& x_2 & x_3 \\x_4 & x_5 & x_6 \\x_7 & x_8 & x_9\\ \end{array}  $$


この和を観測値、ということにする。観測値は全部で16個あるが、
- いくつかが読み取れない
- ノイズが乗っている

場合がある。

この時に、個々のセルの値に近い値を求めることができるだろうか？
できるとすれば、どの程度正しい値が得られるだろうか？

---

この問題を解くために、状況を少し整理しよう。
周辺からの観測値を$b_i$とすると、セル$x_j$との関係は、以下のようになる。


１．左から右にセルを加算する場合

$b_1=x_1+x_2+x_3$

$b_2=x_4+x_5+x_6$

$b_3=x_7+x_8+x_9$

２．右上から左下にセルを加算する場合

$b_4=x_1$

$b_5=x_2 + x_4$

$b_6=x_3 + x_5 + x_7$

$b_7=x_6 + x_8$

$b_8=x_9$

３．上から下にセルを加算する場合

$b_9=x_1 + x_4 + x_7$

$b_{10}=x_2 + x_5 + x_8$

$b_{11}=x_3 + x_6 + x_9$

４．左上から右下にセルを加算する場合

$b_{12}=x_7$

$b_{13}=x_4 + x_8$

$b_{14}=x_1 + x_5 + x_9$

$b_{15}=x_2 + x_6$

$b_{16}=x_3$

さて、これらの和が与えられたときに、個々のセルの値を求めるためには、
どのようにすればよいだろうか？



## NumPyによる解法

${\bf A}\in {\bf R}^{16\times 9}$、${\bf x}\in {\bf R}^9$、 ${\bf b}\in {\bf R}^{16}$、
として、係数行列、セルの内容、観測値に対応する配列を作ろう。
今回は記号演算の必要がないのでNumPyを用いる。

まずは係数の行列をつくる。今回は数が少ないので力業でやってみる。

In [2]:
import numpy as np
A = np.array([[1.0,1,1,0,0,0,0,0,0],
                [0,0,0,1,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,1],
                [1,0,0,0,0,0,0,0,0],
                [0,1,0,1,0,0,0,0,0],
                [1,0,0,0,1,0,0,0,1],
                [0,0,0,0,1,0,0,0,1],
                [0,0,0,0,0,0,0,0,1],
                [1,0,0,1,0,0,1,0,0],
                [0,1,0,0,1,0,0,1,0],
                [0,0,1,0,0,1,0,0,1],
                [0,0,0,0,0,0,1,0,0],
                [0,0,0,1,0,0,0,1,0],
                [1,0,0,0,1,0,0,0,1],
                [0,1,0,0,0,1,0,0,0],
                [0,0,1,0,0,0,0,0,0]])
A

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

型をfloatの配列とするために、先頭の要素だけ$1.0$とした。
この大きさの行列、入力するのはともかく、チェックが意外と大変である。後々、$m\times n$の一般の長方形形状のセルの集合に対応したい。これは、課題としてとっておこう。

では、個々のセルの値を
$$
\begin{array}{ccc}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{array}
$$
と考えて、答えのベクトル${\bf x}$を作ろう。何の変哲もない並びだが、確認にはよかろう。

In [5]:
x = np.array([1.0, 2, 3, 4, 5, 6, 7, 8, 9])
x

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

このセル群を縦、横、斜めから見た時の和を計算するにはどうするのだろうか？

先ほど定義した行列 ${\bf A}$ は、 ${\bf x}$ が入力されたときに和の取り方を示したものなので、
単純に行列とベクトルの掛け算をすればよいはずである。

行列とベクトル、ベクトルとベクトルの積は、NumPyのdot関数で行う。実際にやってみよう。


In [6]:
b=A.dot(x)
b

array([  6.,  15.,  24.,   1.,   6.,  15.,  14.,   9.,  12.,  15.,  18.,
         7.,  12.,  15.,   8.,   3.])

和が出てきた。升目を見ながら加算を行い、このリストと見比べると、結果が意図したものと一致することがわかる。

つまり、この時点で ${\bf Ax}={\bf b}$ のすべての要素がわかった。


最初の問題は ${\bf x}$ を未知として、${\bf b}$ から求めることであった。
これを実現するために、${\bf A}$ の疑似逆行列を求めて、上式の左からかけよう。

疑似逆行列${\bf A}^{-}$は、${\bf A}\in {\bf R}^{m\times n}$とすると
$$  
{\bf A}^{-} = \biggl\{
\begin{array}{cl}
  ({\bf A}^{T}{\bf A})^{-1}{\bf A}^{T} & {\rm if}\ \  m\gt n \\
  {\bf A}^{T}({\bf A}{\bf A}^{T})^{-1} & {\rm if}\ \  m\lt n \\
\end{array}
$$
で定義される。


これを使って、
${\bf x}={\bf A}^{-}{\bf b}$
として、${\bf b}$ から ${\bf x}$を求めよう。NumPyでは、pinv関数で疑似逆行列を求めることができる。

In [7]:
Ainv=np.linalg.pinv(A)
Ainv.dot(b)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

${\bf x}$が元に戻った！

ここで行ったことを、もう一度整理すると,
- 係数行列${\bf A}$を定義
- 観測した数値列${\bf b}$を定義
- ${\bf b}$ 左から疑似逆行列を乗算

することで、${\bf x}$が求まったことになる。



## 逆問題とノイズ


次に、${\bf b}$に少しノイズを加えてみよう。
${\bf b}$を壊さないようにをコピーして${\bf c}$とし、最初の要素に$1.5$を加える。
その後、疑似逆行列を掛けて答えを求める。

In [14]:
c=np.array(b)
c[0] += 1.5
Ainv.dot(c)

array([ 1.2293578 ,  2.29095675,  3.41153342,  3.93381389,  4.95412844,
        5.83224115,  6.98296199,  7.9750983 ,  8.91284404])

一つ値を変えただけで、全体の値が少しずつ変わってしまうことがわかる。
四捨五入をするとどうだろう？

In [15]:
np.rint(Ainv.dot(c))

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

元の${\bf x}$と同じであることがわかる。

今度は異なる部分を変えてみよう。$4$番目の要素に同じ値$1.5$を加えてみよう。


In [18]:
d=np.array(b)
d[3]+=1.5
np.rint(Ainv.dot(d))

array([ 2.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

同じノイズを加えたのにもかかわらず、今度は${\bf x}$の値が変わっってしまった。

どうやら、ノイズに弱い観測値と、ノイズに強い観測値があるようである。

## 欠損データ

欠損データを扱うことができるようにしてみよう。

${\bf b}$の要素に NaN (not a number)が入っている場合には、このデータが欠損していると考えるとする。

この時、欠損データの位置に対応する係数行列${\bf A}$の行を削除し、${\bf b}$の欠損データ自体も消してしまうこととする。
こうすれば、欠損データとそれに対応する方程式がなかったことになる。

NumPy配列のdelete関数を使ってこれを実現するが、添え字を後ろから処理していることに注意しよう。

In [36]:
def magic_square3(coef, b):
    A = np.array(coef)
    s = np.array(b)
    for i in reversed(range(b.shape[0])):
        if(np.isnan(b[i])):
            A=np.delete(A, i, 0)
            s=np.delete(s,i)
    return A,s

それでは、これを使ってみよう。

${\bf b}$ の先頭から$9$つの要素をバッサリと消して、
対応する${\bf A}$の行列を求める。

In [69]:
e=np.array(b)
for i in range(0,8):
    e[i]=np.nan
A2, e2 = magic_square3(A,e)
A2, e2, e

(array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
        [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 array([ 12.,  15.,  18.,   7.,  12.,  15.,   8.,   3.]),
 array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  12.,  15.,  18.,
          7.,  12.,  15.,   8.,   3.]))

得られた${\bf A}_2$と${\bf e}_2$を使って、答えを求めよう。

In [48]:
np.linalg.pinv(A2).dot(e2)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

答えが一致した！

この時、行列のランクは以下のようにして求めることができる。

In [50]:
np.linalg.matrix_rank(A2)

7

$9$つの未知数に対して、ランクが$7$であるので、方程式を解いただけでは解は定まらないはずである。

しかし、疑似逆行列を使うことによって、解を得ることができた！

もっと観測データを減らすとどうなるだろうか？

In [70]:
f=np.array(b)
for i in range(0,10):
    f[i]=np.nan
A3, f2 = magic_square3(A,f)
f, np.rint(np.linalg.pinv(A3).dot(f2))

(array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  18.,
          7.,  12.,  15.,   8.,   3.]),
 array([ 3.,  1.,  3.,  6.,  3.,  7.,  7.,  6.,  8.]))

さすがに、データ数が$6$つだと少なすぎるようである。

欠損している観測値の数や位置と、推定するセルの値には、どのような関係があるのだろうか？

## まとめ

ここでは、逆問題の例として、正方形状に並べられたセルの値を、周辺からの観測値によって推定する問題を取り上げた。
また、この問題をNumPyを使って解く方法と、ノイズと欠損値による解への影響を調べる方法を示した。

## 宿題

国内の固定電話の電話番号は

|国内プレフィックス| 市外局番・市内局番 | 加入者番号|
|---|---|---|
|0| 計5桁 | 4桁 |

で表現されている。

Web上で公開されている国内の固定電話の電話番号を3つ選び、
それぞれの番号に対してnanの要素がなるべく増えるように以下の操作を行う。

1. 国内プレフィックスをのぞいた9桁の電話番号を左上から右下に配置する。
2. 観測値を生成する
2. 生成した観測値のいくつかの要素をnanにする
3. 元の電話番号に戻るかを確認する

つまり、電話番号を最小の要素数で表現する、ということである。
この表現を得た後、以下の問いに答えよ。


問題１
---
それぞれの電話番号と、電話番号に対してなるべくnanの数が多くなるように
作成した観測値のベクトルを報告せよ。

問題２
---
それぞれの電話番号と、nanに置き換えた観測値の位置は同じだろうか？異なるだろうか？

同じ・あるいは異なる理由を、自分なりに考えて答えよ。この理由は数学的に厳密でなくてもよい。

問題３
---
それぞれの電話番号に対する、要素の数を減らす前と減らした後の観測値のベクトルを考える。
これらの同じ位置の要素に同じノイズを加えてみよ。また、大きさも変えてみよ。

加えたノイズとその影響を説明し、なぜそうなるのか理由を自分なりに考えて答えよ。
この理由は数学的に厳密でなくてもよい。

問題４
---
以下は電話番号から離れて、逆魔法陣問題そのものに関する設問である。
問いに答えよ。

1. $n\times n \ \ (n=1,2,\cdots )$ のセルに対する観測値はいくつになるだろうか？
1. $n$ を増やしていったときに、セルの数と観測値の数はどのような関係になるだろうか？
また、$n$が増加したときに、逆魔法陣問題は問題として成り立つだろうか？

問題５
---

逆魔法陣問題を拡張し、以下のいずれができるような命令列を答えよ。
1. $5\times 5$のセルへの対応
1. $n\times n$のセルへの対応
1. $m\times n$のセルへの対応

参考文献
---
1. 小國健二，「応用例で学ぶ逆問題と計測」，オーム社，2011年, pp. 3-19
1. 総務省，「電話番号に関するQ&A」，< http://www.soumu.go.jp/main_sosiki/joho_tsusin/top/tel_number/q_and_a.html#q2 >，（参照2004-8-4）

